https://oauth.vk.com/authorize?client_id=6052784&scope=271502&redirect_uri=https://oauth.vk.com/blank.html&display=page&v=5.63&response_type=token

In [1]:
# copy token here
token = "b0eccfa8b4fe3dca107904cdcae88aee85291dfb37b141bf552fb8d625986dc3b0bbadf1384f4a9051abc"

In [2]:
poll_group_id = "-172053584"

# VK API

In [3]:
import vk
from vk.exceptions import VkAPIError

In [4]:
session = vk.Session(token)
api = vk.API(session, v="5.74")

# Getting wall data

### Creating data dir if it doesn't exist

In [5]:
import os

In [6]:
os.makedirs("data", exist_ok=True)

### Requesting first 100

In [7]:
kwargs = {"owner_id": poll_group_id, "count": "100"}

group_wall_json = api.wall.get(**kwargs)

print("got {} items".format(len(group_wall_json["items"])))

got 100 items


### Requesting the rest

In [8]:
import time

while len(group_wall_json["items"]) < group_wall_json["count"]:
    kwargs.update({
        "offset": len(group_wall_json["items"])
    })
    extra_group_wall_json = api.wall.get(**kwargs)
    group_wall_json["items"].extend(extra_group_wall_json["items"])
    print("Number of items now: {}".format(len(group_wall_json["items"])))
    time.sleep(3)

Number of items now: 200
Number of items now: 300
Number of items now: 400
Number of items now: 500
Number of items now: 600
Number of items now: 700
Number of items now: 800
Number of items now: 900
Number of items now: 1000
Number of items now: 1064


### Saving the result

In [9]:
import json

In [10]:
with open("data/group_wall.json", "w") as f:
    json.dump(group_wall_json, f)

# Getting poll voters

In [11]:
with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [12]:
def get_poll(wall_post):
    if "attachments" not in wall_post:
        return None
    for attachment in wall_post["attachments"]:
        if attachment["type"] != "poll":
            continue
        poll = attachment["poll"]
        return poll
    return None

In [13]:
def vote_in_poll(api, poll):
    answer_id = poll["answers"][0]["id"]
    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "answer_ids": [answer_id],
    }
    
    return api.polls.addVote(**kwargs)

In [14]:
for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
    poll_filename = "data/poll_{}.json".format(poll["id"])

    if os.path.isfile(poll_filename):
        continue
    
    if poll["answer_id"] == 0:
#         print("Answering result: {}".format(vote_in_poll(api, poll)))
        print("needs answering: https://vk.com/asurveys?w=poll-172053584_{}".format(poll["id"]))
        continue

    kwargs = {
        "poll_id": poll["id"],
        "owner_id": poll["owner_id"],
        "count": 1000,
        "answer_ids": list(map(lambda answer: answer["id"], poll["answers"])),
        "fields": "nickname, screen_name, sex, photo, photo_medium, photo_big"
    }
    
    try:
        poll_voters = api.polls.getVoters(**kwargs)
    except VkAPIError as vk_error:
        print(vk_error)
        print(poll)
        time.sleep(1)
        continue
    with open(poll_filename, "w") as f:
        json.dump(poll_voters, f)
    if (i + 1) % 20 == 0:
        print("Processed {}/{} polls".format(i + 1, len(group_wall_json["items"])))
    time.sleep(5)

needs answering: https://vk.com/asurveys?w=poll-172053584_306718174


# Processing and squeezing data

In [15]:
# import json
# with open("data/poll_307034744.json", "r") as f:
#     poll = json.load(f)
# poll

### Building main mappings:

In [17]:
poll_id_to_answer_id_to_user_ids = {}
user_id_to_user = {}

for filename in os.listdir("data"):
    if "poll_" not in filename:
        continue
    poll_id = int(filename.split("poll_")[1].split(".")[0])
    poll_id_to_answer_id_to_user_ids[poll_id] = {}

    with open("data/" + filename, "r") as f:
        poll = json.load(f)
    
    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        voted_users = answer["users"]["items"]

        answer_id = answer["answer_id"]
        voted_user_ids = list(map(lambda voted_user: voted_user["id"], voted_users))

        for voted_user in voted_users:
            user_id_to_user[voted_user["id"]] = voted_user
        
        poll_id_to_answer_id_to_user_ids[poll_id][answer_id] = voted_user_ids

#         answer_id_to_user = list(map(lambda vote: (vote["id"], vote), ))
#         all_ids_to_users.update(dict(answer_id_to_user))
#         all_ids.extend(map(lambda item: item[0], answer_id_to_user))

In [18]:
with open("data/user_id_to_user.json", "w") as f:
    json.dump(user_id_to_user, f)

In [19]:
with open("data/poll_id_to_answer_id_to_user_ids.json", "w") as f:
    json.dump(poll_id_to_answer_id_to_user_ids, f)

In [ ]:
# from collections import Counter
# cnt = Counter(all_ids)

In [ ]:
# for user_id in cnt:
#     all_ids_to_users[user_id]["num_votes"] = cnt[user_id]

In [ ]:
# with open("data/users.json", "w") as f:
#     json.dump(all_ids_to_user, f)

### Building "sparse" matrix.

In [ ]:
# polls = []
# for filename in os.listdir("data"):
#     if "poll_" not in filename:
#         continue
#     poll_id = int(filename.split("poll_")[1].split(".")[0])
#     with open("data/" + filename, "r") as f:
#         poll = json.load(f)
    
#     if not isinstance(poll, list):
#         print("error")
    
#     answers_to_ids = []
#     for answer in poll:
#         answers_to_ids.append(list(map(lambda vote: vote["id"], answer["users"]["items"])))
#     polls.append(answers_to_ids)

Now we have variable 'polls' which is [POLL_ID \* ANSWER_ID \* USER] where ids are not real but just indexation.

In [ ]:
# with open("data/vote_matrix.json", "w") as f:
#     json.dump(polls, f)

# Now let's make some stats

# YESNO

### Let's find all YesOrNo questions

In [20]:
import json

with open("data/group_wall.json", "r") as f:
    group_wall_json = json.load(f)

In [21]:
# group_wall_json["items"][0]

In [22]:
def clear_text(text):
    return ''.join(filter(str.isalpha, text.lower()))

In [23]:
yes_no_polls = {}

for i, wall_post in enumerate(group_wall_json["items"]):
    poll = get_poll(wall_post)
    if poll is None or poll["anonymous"]:
        continue
        
    answer_texts = list(map(lambda answer: clear_text(answer["text"]), poll["answers"]))
    if "да" in answer_texts and "нет" in answer_texts:
        
        yes_no_polls[poll["id"]] = {
            "id": poll["id"],
            "yes_id": poll["answers"][answer_texts.index("да")]["id"],
            "no_id": poll["answers"][answer_texts.index("нет")]["id"]
        }
        

In [24]:
print("Number of yes/no questions: {}".format(len(yes_no_polls)))

Number of yes/no questions: 145


In [28]:
os.makedirs("datax/")

### Now let's find out who are YESNO people

In [25]:
from collections import defaultdict

yes_cnt = defaultdict(int)
no_cnt = defaultdict(int)

for yes_no_poll in yes_no_polls.values():
    poll_filepath = "data/poll_{}.json".format(yes_no_poll["id"])

    try:
        with open(poll_filepath, "r") as f:
            poll = json.load(f)
    except FileNotFoundError as error:
        print(error)

    if not isinstance(poll, list):
        print("error")
    
    answers_to_ids = []
    for answer in poll:
        if answer["answer_id"] == yes_no_poll["yes_id"]:
            for user in answer["users"]["items"]:
                yes_cnt[user["id"]] += 1
        if answer["answer_id"] == yes_no_poll["no_id"]:
            for user in answer["users"]["items"]:
                no_cnt[user["id"]] += 1


In [29]:
def get_max_cnt(cnt_dict):
    return list(filter(lambda key: cnt_dict[key] == max(cnt_dict.values()), cnt_dict))

In [30]:
stat = {
    "stat_name": "Человек-ДА",
    "stat_description": "Тот самый разборчивый человек, что почти всегда жмет ДА",
    "user_ids": get_max_cnt(yes_cnt)
}
with open("datax/stat_yes_man.json", "w") as f:
    json.dump(stat, f)

In [31]:
stat = {
    "stat_name": "Человек-НЕТ",
    "stat_description": "Тот самый разборчивый человек, что почти всегда жмет НЕТ",
    "user_ids": get_max_cnt(no_cnt)
}

with open("datax/stat_no_man.json", "w") as f:
    json.dump(stat, f)

# Make 1x1 correlation

In [32]:
import json

with open("data/poll_id_to_answer_id_to_user_ids.json", "r") as f:
    poll_id_to_answer_id_to_user_ids = json.load(f)

In [43]:
a = {1, 2, 5}

a.update([1, 2, 3, 4])

In [46]:
user_id_to_user_id_to_ncommon_polls = {}
for poll_id in poll_id_to_answer_id_to_user_ids:
    poll_users = set()

    answer_id_to_user_ids = poll_id_to_answer_id_to_user_ids[poll_id]
    for answer_id in answer_id_to_user_ids:
        user_ids = answer_id_to_user_ids[answer_id]
        poll_users.update(user_ids)

    for user_id_1 in poll_users:
        if user_id_1 not in user_id_to_user_id_to_ncommon_polls:
            user_id_to_user_id_to_ncommon_polls[user_id_1] = {}

        for user_id_2 in poll_users:
            if user_id_2 not in user_id_to_user_id_to_ncommon_polls[user_id_1]:
                user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2] = 0
            
            user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2] += 1

In [47]:
user_id_to_user_id_to_ncommon_answer = {}
for poll_id in poll_id_to_answer_id_to_user_ids:
    answer_id_to_user_ids = poll_id_to_answer_id_to_user_ids[poll_id]
    for answer_id in answer_id_to_user_ids:
        user_ids = answer_id_to_user_ids[answer_id]
        for user_id_1 in user_ids:
            user_id_to_user_id_to_ncommon_answer[user_id_1] = {}
            for user_id_2 in user_ids:
                if user_id_2 not in user_id_to_user_id_to_ncommon_answer[user_id_1]:
                    user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2] = 0
                user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2] += 1

In [53]:
MIN_NUMBER_OF_COMMON_POLLS = 50

user_id_to_user_id_to_correlation = {}

for user_id_1 in user_id_to_user_id_to_ncommon_polls:
    for user_id_2 in user_id_to_user_id_to_ncommon_polls[user_id_1]:
        
        ncommon_polls = user_id_to_user_id_to_ncommon_polls[user_id_1][user_id_2]
        ncommon_answer = user_id_to_user_id_to_ncommon_answer[user_id_1][user_id_2]
        
        if ncommon_polls > MIN_NUMBER_OF_COMMON_POLLS:
            if user_id_1 not in user_id_to_user_id_to_correlation:
                user_id_to_user_id_to_correlation[user_id_1] = {}
                if user_id_2 not in user_id_to_user_id_to_correlation[user_id_1]:
                    user_id_to_user_id_to_correlation[user_id_1][user_id_2] = 1.0 * ncommon_answer / ncommon_polls


a = 153727
b = 1448133


In [54]:
with open("data/user_id_to_user_id_to_correlation.json", "w") as f:
    json.dump(user_id_to_user_id_to_correlation, f)